In [25]:
# load library
suppressPackageStartupMessages(library(org.Hs.eg.db))
suppressPackageStartupMessages(library(viper))
suppressPackageStartupMessages(library(fgsea))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(readr))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(readr))
suppressPackageStartupMessages(library(AnnotationDbi))
hs <- org.Hs.eg.db
select<-AnnotationDbi::select

In [ ]:
#######################################
## 0. preprocessing - normalizedData ##
#######################################
filtered <- function(cntData, cellName){
  veh <- grep(cellName,colnames(cntData))[grep('veh',colnames(cntData)[grep(cellName,colnames(cntData))])]
  tram <- grep(cellName,colnames(cntData))[grep('tram',colnames(cntData)[grep(cellName,colnames(cntData))])]
  df <-cntData[,c(veh,tram)]
  return(df)}

normalizedData <- read_csv('/data/Gitcode/QuantSeq_normalized.csv')
geneName<-list(normalizedData$`Gene symbol`)
normalizedData<-aggregate(normalizedData, by=geneName, FUN=max)
rownames(normalizedData)<-normalizedData$Group.1
normalizedData<-normalizedData[,-c(1)]

sw1573edata<-filtered(normalizedData,'SW1573')
a549edata<-filtered(normalizedData,'A549')
h2009edata<-filtered(normalizedData,'H2009')
h1792edata<-filtered(normalizedData,'H1792')

sw1573edata2 <- sw1573edata[matrixStats::rowVars(as.matrix(sw1573edata))>0.1,]
a549edata2 <- a549edata[matrixStats::rowVars(as.matrix(a549edata))>0.1,]
h2009edata2 <- h2009edata[matrixStats::rowVars(as.matrix(h2009edata))>0.1,]
h1792edata2 <- h1792edata[matrixStats::rowVars(as.matrix(h1792edata))>0.1,]

In [22]:
##############################
## 1. Data analysis : VIPER ##
##############################
library(aracne.networks)
data(regulonluad)
# load('/data/Gitcode/TCGAregulon.RData')

# =========================================
# veh vs trametinib
# =========================================

sw1573edata2_24 <- sw1573edata2[,grep('24h',colnames(sw1573edata2))]
a549edata2_24 <- a549edata2[,grep('24h',colnames(a549edata2))]
h2009edata2_24 <- h2009edata2[,grep('24h',colnames(h2009edata2))]
h1792edata2_24 <- h1792edata2[,grep('24h',colnames(h1792edata2))]

sw1573edata2_3 <- sw1573edata2[,grep('3d',colnames(sw1573edata2))]
a549edata2_3 <- a549edata2[,grep('3d',colnames(a549edata2))]
h2009edata2_3 <- h2009edata2[,grep('3d',colnames(h2009edata2))]
h1792edata2_3 <- h1792edata2[,grep('3d',colnames(h1792edata2))]
makeExpv_eachTime <- function(edata){
  df <- as.matrix(edata)
  # gene symbol to entrez id
  dfe <- bind_cols(tibble(SYMBOL=rownames(df)), df %>% as_tibble) %>% left_join(AnnotationDbi::select(hs, 
                                                                                                      keys = rownames(df),
                                                                                                      columns = c("ENTREZID", "SYMBOL"),
                                                                                                      keytype = "SYMBOL") %>% as_tibble, by='SYMBOL') %>% drop_na
  
  mtx <- as.matrix(dfe[,2:(ncol(dfe)-1)])
  rownames(mtx) <- dfe$ENTREZID
  dset <- ExpressionSet(assayData = as.matrix(mtx))
  featureNames(dset) <- rownames(mtx)
  sampleNames(dset) <- colnames(mtx)
  
  fc<-as.factor(ifelse(grepl('tram',colnames(mtx)),'tram','veh'))
  names(fc)<-colnames(mtx)
  dset$description <- fc
  print(dset$description)
  
  sig <- viperSignature(exprs(dset)[,grep('tram',colnames(dset))],exprs(dset)[,grep('veh',colnames(dset))],method=c('ttest'),per=1000)
  vpres <- viper(sig,regulonluad,verbose=FALSE)
  vpres <- data.frame(rowMeans(vpres))
  colnames(vpres)<-'activity'
  expv <- data.frame(ENTREZID=rownames(vpres), vpres) %>% as_tibble %>% left_join(AnnotationDbi::select(hs, 
                                                                                                        keys = rownames(vpres),
                                                                                                        columns = c("ENTREZID", "SYMBOL","UNIPROT"),
                                                                                                        keytype = "ENTREZID") %>% as_tibble, by='ENTREZID') %>% drop_na
  
  
  return (expv)
}

sw1573_tram_24h <- makeExpv_eachTime(sw1573edata2_24)
a549_tram_24h <- makeExpv_eachTime(a549edata2_24)
h2009_tram_24h <- makeExpv_eachTime(h2009edata2_24)
h1792_tram_24h <- makeExpv_eachTime(h1792edata2_24)

sw1573_tram_3d <- makeExpv_eachTime(sw1573edata2_3)
a549_tram_3d <- makeExpv_eachTime(a549edata2_3)
h2009_tram_3d <- makeExpv_eachTime(h2009edata2_3)
h1792_tram_3d <- makeExpv_eachTime(h1792edata2_3)

sw1573_tram <- (sw1573_tram_24h %>% dplyr::select(SYMBOL, activity) %>% unique()) %>% left_join((sw1573_tram_3d %>% dplyr::select(SYMBOL, activity) %>% unique()), by='SYMBOL')
a549_tram <- (a549_tram_24h %>% dplyr::select(SYMBOL, activity) %>% unique()) %>% left_join((a549_tram_3d %>% dplyr::select(SYMBOL, activity) %>% unique()), by='SYMBOL')
h2009_tram <- (h2009_tram_24h %>% dplyr::select(SYMBOL, activity) %>% unique()) %>% left_join((h2009_tram_3d %>% dplyr::select(SYMBOL, activity) %>% unique()), by='SYMBOL')
h1792_tram <- (h1792_tram_24h %>% dplyr::select(SYMBOL, activity) %>% unique()) %>% left_join((h1792_tram_3d %>% dplyr::select(SYMBOL, activity) %>% unique()), by='SYMBOL')


'select()' returned 1:many mapping between keys and columns



[1] veh  veh  veh  tram tram tram
Levels: tram veh


Warning message:
“Not enough samples to compute null model by sample permutation, gene permutation will be used instead”
'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns



[1] veh  veh  veh  tram tram tram
Levels: tram veh


Warning message:
“Not enough samples to compute null model by sample permutation, gene permutation will be used instead”
'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns



...

Warning message:
“Not enough samples to compute null model by sample permutation, gene permutation will be used instead”
'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns



[1] veh  veh  veh  tram tram tram
Levels: tram veh


Warning message:
“Not enough samples to compute null model by sample permutation, gene permutation will be used instead”
'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns



[1] veh  veh  veh  tram tram tram
Levels: tram veh


Warning message:
“Not enough samples to compute null model by sample permutation, gene permutation will be used instead”
'select()' returned 1:many mapping between keys and columns

